In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from cnn_finetune import make_model

from net import AGNet
from loss import AGLoss

from torch.autograd import Variable
from datagen import ListDataset

import os

In [2]:
batch_size = 32
test_batch_size = 64
epochs = 50
learning_rate = 0.01
momentum = 0.9
dropout = 0.2
start_epoch = 0
best_correct = 0
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [3]:
transform_train = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.RandomCrop(150, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)
    )
])
trainset = ListDataset(root='../data/UTKFace/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=8)

transform_test = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225))
])
testset = ListDataset(root='../data/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=8)

In [4]:
net = AGNet()
net.cuda()
criterion = AGLoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=1e-4)

In [5]:
def accuracy(age_preds, age_targets, gender_preds, gender_targets):
# def accuracy(gender_preds, gender_targets):

    AGE_TOLERANCE = 2
    age_prob = F.softmax(age_preds)
    age_predict_batch = torch.from_numpy(np.array([torch.argmax(age_prob[i]) + 1 for i in range(0, 128)]))
    print(type(age_predict_batch))
    
#     age_expect = torch.sum(Variable(torch.arange(1, 117)).cuda() * age_prob, 1)
    age_correct = ((age_predict_batch - age_targets).abs() < AGE_TOLERANCE).int().sum().cpu().data[0]
    age_correct = ((age_expect - age_targets.int()).abs() < AGE_TOLERANCE).int().sum().cpu().data[0]
    
    gender_preds = F.sigmoid(gender_preds)
    gender_preds = (gender_preds > 0.5).int()
    gender_correct = (gender_preds == gender_targets.int()).int().cpu().sum().data[0]
#     return gender_correct
    return age_correct, gender_correct

In [6]:
def train(epoch):
    net.train()
    train_loss = 0
    total = 0
    age_correct = 0
    gender_correct = 0
    for batch_idx, (inputs, age_targets, gender_targets) in enumerate(trainloader):
        inputs = Variable(inputs.cuda())
        age_targets = Variable(age_targets.cuda())
        gender_targets = Variable(gender_targets.cuda())
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # 
        age_preds, gender_preds = net(inputs)
        loss = criterion(age_preds, age_targets, gender_preds, gender_targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data[0]
        
        age_correct_i, gender_correct_i = accuracy(age_preds,
                                                  age_targets, gender_preds,
                                                  gender_targets)
#         gender_correct_i = accuracy(gender_preds, gender_targets)
        age_correct += age_correct_i
        gender_correct += gender_correct_i
        total += len(inputs)
#         print('train_loss: %f | avg_loss: %f | gender_precise: %f (%d/%d) [%d/%d]'
#              % (loss.data[0], train_loss/(batch_idx+1),
#                100.*gender_correct/total, gender_correct, total,
#                batch_idx+1, len(trainloader)))
        print('train_loss: %f | avg_loss: %f | age_prec: %f (%d/%d) | gender_prec: %f (%d/%d)  [%d/%d]'  \
            % (loss.data[0], train_loss/(batch_idx+1),      \
               100.*age_correct/total, age_correct, total,  \
               100.*gender_correct/total, gender_correct, total,    \
               batch_idx+1, len(trainloader)))

In [7]:
# Test
def test(epoch):
    print('\nTest')
    net.eval()
    test_loss = 0
    total = 0
    age_correct = 0
    gender_correct = 0
    for batch_idx, (inputs, age_targets, gender_targets) in enumerate(testloader):
        inputs = Variable(inputs.cuda())
        age_targets = Variable(age_targets.cuda())
        gender_targets = Variable(gender_targets.cuda())

#         age_preds, gender_preds = net(inputs)
        age_preds, gender_preds = net(inputs)
#         loss = criterion(age_preds, age_targets, gender_preds, gender_targets)
#         loss = criterion(gender_preds, gender_targets)

#         test_loss += loss.data[0]
#         gender_correct_i = accuracy(gender_preds, gender_targets)
        age_correct_i, gender_correct_i = accuracy(
            age_preds, age_targets, gender_preds, gender_targets)
    
        age_correct += age_correct_i
        gender_correct += gender_correct_i
        total += len(inputs)
#         print('gender_prec: %f (%d/%d) [%d/%d]' % (100.*gender_correct/total, gender_correct, total, batch_idx+1, len(testloader)))
        print('test_loss: %f | avg_loss: %f | age_prec: %f (%d/%d) | gender_prec: %f (%d/%d)  [%d/%d]' \
            % (loss.data[0], test_loss/(batch_idx+1),      \
               100.*age_correct/total, age_correct, total,  \
               100.*gender_correct/total, gender_correct, total, \
               batch_idx+1, len(testloader)))

    # Save checkpoint
    global best_correct
    if age_correct + gender_correct > best_correct:
#     if gender_correct > best_correct:
        print('Saving..')
        best_correct = gender_correct + age_correct
        state = {
            'net': net.state_dict(),
            'correct': best_correct,
            'epoch': epoch,
        }
        if not os.path.isdir('../weights/checkpoint'):
            os.mkdir('../weights/checkpoint')
        torch.save(state, '..weights/checkpoint/ckpt_age_gender.pth')

In [8]:
for epoch in range(start_epoch, start_epoch+10):
    print("Number epoch: {}".format(epoch))
    train(epoch)
    test(epoch)

Number epoch: 0
torch.Size([128, 116])
torch.Size([128])
torch.Size([128])
torch.Size([128])


RuntimeError: smooth_l1_loss_forward is not implemented for type torch.cuda.IntTensor

In [ ]:
import torch
import numpy as np

In [ ]:
x = torch.randn(128, 116)

In [ ]:
x.shape

In [ ]:
x

In [ ]:
x.